In [59]:
from glob import glob

gpt35 = glob("./ds1000_gpt35_predictions/gpt35/*/Completion/*/*.py")
react_gpt35 = glob("./ds1000_gpt35_predictions/react-gpt35/*/Completion/*/*.py")

In [60]:
len(gpt35), len(react_gpt35)

(1000, 1000)

In [61]:
gpt35 = sorted(gpt35)
react_gpt35 = sorted(react_gpt35)
total_changes = []
for i in range(len(gpt35)):
    with open(gpt35[i], "rt") as file:
        text1 = file.read()
    with open(react_gpt35[i], "rt") as file:
        text2 = file.read()
    if(text1 != text2):
        total_changes.append((gpt35[i], react_gpt35[i]))

In [62]:
print(len(total_changes))

400


In [63]:
total_changes[:10]

[('./ds1000_gpt35_predictions/gpt35/Matplotlib/Completion/q0/0.py',
  './ds1000_gpt35_predictions/react-gpt35/Matplotlib/Completion/q0/0.py'),
 ('./ds1000_gpt35_predictions/gpt35/Matplotlib/Completion/q1/0.py',
  './ds1000_gpt35_predictions/react-gpt35/Matplotlib/Completion/q1/0.py'),
 ('./ds1000_gpt35_predictions/gpt35/Matplotlib/Completion/q100/0.py',
  './ds1000_gpt35_predictions/react-gpt35/Matplotlib/Completion/q100/0.py'),
 ('./ds1000_gpt35_predictions/gpt35/Matplotlib/Completion/q101/0.py',
  './ds1000_gpt35_predictions/react-gpt35/Matplotlib/Completion/q101/0.py'),
 ('./ds1000_gpt35_predictions/gpt35/Matplotlib/Completion/q103/0.py',
  './ds1000_gpt35_predictions/react-gpt35/Matplotlib/Completion/q103/0.py'),
 ('./ds1000_gpt35_predictions/gpt35/Matplotlib/Completion/q104/0.py',
  './ds1000_gpt35_predictions/react-gpt35/Matplotlib/Completion/q104/0.py'),
 ('./ds1000_gpt35_predictions/gpt35/Matplotlib/Completion/q106/0.py',
  './ds1000_gpt35_predictions/react-gpt35/Matplotlib/Com

In [3]:
import os
import json
from pathlib import Path
from collections import defaultdict
import numpy as np
from glob import glob
from ds1000 import ScoreRecord
from typing import Dict, List, Union


def load_record(record_path: Union[str, Path], libs: List[str] = None) -> dict:
    """
    Load json files into ScoreRecord

    Input:
    `record_path` is the path where json files are stored
    `libs` is a list of str like ["Pandas", "Numpy"], specifying which libraries' score records to load

    Return:
    A dict `score`, as score["Pandas"][0] is a `ScoreRecord` used for accuracy analysis
    """
    score = defaultdict(list)
    record_path = Path(record_path)
    if not libs:
        libs = os.listdir(record_path)
    for lib in libs:
        num_questions = len(glob("{}/{}/*.json".format(record_path, lib)))
        
        for question_id in range(num_questions):
            with open(
                record_path / lib / f"{question_id}.json", "r", encoding="UTF-8"
            ) as f:
                t = json.load(f)
            score[lib].append(
                ScoreRecord(
                    library=lib,
                    problem_id=t["id"],
                    perturbation_type=t["perturbation_type"],
                    perturbation_origin_id=t["original_id"],
                    test_results=t["test_results"],
                )
            )
    return score
libs = [
                "Pandas",
                "Numpy",
                "Matplotlib",
                "Tensorflow",
                "Scipy",
                "Sklearn",
                "Pytorch",
            ]
score_by_lib_baseline = load_record("./ds1000_gpt35_predictions/gpt35_Completion_result_cache/", libs)
score_by_lib_re = load_record("./ds1000_gpt35_predictions/react-gpt35_Completion_result_cache/", libs)
# os.listdir("./ds1000_gpt35_predictions/gpt35_Completion_result_cache/Pandas")

In [7]:
remain_incorrect_example= []
for lib in libs:
    
    corrected = 0
    incorrected = 0
    remain_correct = 0
    remain_incorrect = 0
    
    
    baseline_problemset = sorted(score_by_lib_baseline[lib], key=lambda x: x.problem_id)
    re_problemset = sorted(score_by_lib_re[lib], key=lambda x: x.problem_id)
    
    for i in range(len(baseline_problemset)):
        baseline_res = sum(baseline_problemset[i].test_results)
        re_res = sum(re_problemset[i].test_results)
        
        if(re_res == baseline_res == 0):
            remain_incorrect +=1
            remain_incorrect_example.append(baseline_problemset[i])
        elif(re_res == baseline_res and re_res > 0):
            remain_correct += 1
        elif(re_res < baseline_res):
            incorrected +=1
        elif(re_res > baseline_res):
            corrected += 1
    print("Corrected Incorrected remain_correct remain_incorrect total")
    print("{} & {} & {} & {} & {} & {}".format(lib, corrected, incorrected, remain_correct, remain_incorrect, len(baseline_problemset)))

Corrected Incorrected remain_correct remain_incorrect total
Pandas & 2 & 2 & 80 & 207 & 291
Corrected Incorrected remain_correct remain_incorrect total
Numpy & 4 & 7 & 99 & 110 & 220
Corrected Incorrected remain_correct remain_incorrect total
Matplotlib & 2 & 3 & 103 & 47 & 155
Corrected Incorrected remain_correct remain_incorrect total
Tensorflow & 0 & 0 & 16 & 29 & 45
Corrected Incorrected remain_correct remain_incorrect total
Scipy & 2 & 2 & 39 & 63 & 106
Corrected Incorrected remain_correct remain_incorrect total
Sklearn & 2 & 34 & 18 & 61 & 115
Corrected Incorrected remain_correct remain_incorrect total
Pytorch & 0 & 20 & 9 & 39 & 68


In [8]:
remain_incorrect_example

[ScoreRecord(library='Pandas', problem_id=1, perturbation_type='Difficult-Rewrite', perturbation_origin_id=0, test_results=[False]),
 ScoreRecord(library='Pandas', problem_id=2, perturbation_type='Origin', perturbation_origin_id=2, test_results=[False]),
 ScoreRecord(library='Pandas', problem_id=3, perturbation_type='Semantic', perturbation_origin_id=2, test_results=[False]),
 ScoreRecord(library='Pandas', problem_id=4, perturbation_type='Surface', perturbation_origin_id=2, test_results=[False]),
 ScoreRecord(library='Pandas', problem_id=5, perturbation_type='Difficult-Rewrite', perturbation_origin_id=2, test_results=[False]),
 ScoreRecord(library='Pandas', problem_id=6, perturbation_type='Difficult-Rewrite', perturbation_origin_id=2, test_results=[False]),
 ScoreRecord(library='Pandas', problem_id=7, perturbation_type='Origin', perturbation_origin_id=7, test_results=[False]),
 ScoreRecord(library='Pandas', problem_id=8, perturbation_type='Semantic', perturbation_origin_id=7, test_resu

In [2]:
import os
import sys
from dotenv import load_dotenv
import json
import numpy as np
import pandas as pd
import openai


load_dotenv("../.env")

sys.path.append("./DS-1000")
from ds1000 import DS1000Dataset

ds_data = DS1000Dataset("./DS-1000/ds1000_example/", mode="Completion") # loads all questions into RAM

prompt = """
Problem:
How do I get the dimensions of an array? For instance, this is (2, 2):
a = np.array([[1,2],[3,4]])

A:
<code>
import numpy as np
a = np.array([[1,2],[3,4]])
</code>
result = ... # put solution in this variable
BEGIN SOLUTION
<code>

"""

client = openai.AzureOpenAI(
    api_key=os.environ["OPENAI_API_KEY"],  
    api_version=os.environ["OPENAI_API_VERSION"],
    azure_endpoint=os.getenv("OPENAI_ENDPOINT")
)
# response = client.completions.create(
#                 model=os.environ["OPENAI_DEPLOYMENT"],
#                 prompt=prompt,
#                 temperature=0.0,
#                 max_tokens=1024,
#                 top_p=0.95,
#                 frequency_penalty=0,
#                 presence_penalty=0,
#                 stop=["</code>", "# SOLUTION END"],
#                 logprobs=1,
#             )



2024-04-15 05:36:05.788306: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-15 05:36:10.932170: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 05:36:14.612571: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-15 05:36:19.007825: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [9]:
# from langchain.agents import AgentExecutor, create_react_agent
# from langchain_openai import AzureOpenAI
# from langchain_core.prompts import PromptTemplate
# from langchain_experimental.tools import PythonREPLTool

from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

# import dspy

python_repl = PythonREPL()


# lm = dspy.AzureOpenAI(
#     model="stic-isl-gpt35",
#     api_key=os.getenv('OPENAI_API_KEY'),
#     api_base=os.getenv('OPENAI_ENDPOINT'),
#     model_type="chat",
#     api_version=os.getenv('OPENAI_API_VERSION'))

# dspy.settings.configure(lm=lm)

is_correct_prompt_template = """
The following is a Data Science python code generation problem.
You are provided with one possible answer to the problem
and the output of the code using the provided answer.
You need to determine if the answer is correct or not based on the input. 

Problem: The original problem statement
Possible Answer: Possible solution to the problem 
Generated Output: Output generated using the possible answer
Is Correct: Is the answer correct or not. need to be YES or NO

Problem: {problem}
Possible Answer: 
{answer} 
Generated Output: 
{output}
Is Correct: """

modify_answer_prompt_template = """
The following is a Data Science python code generation problem.
You are provided with one incorrect answer to the problem
and the output of the code using the provided incorrect answer.
Write the corrected answer and import any necessary libraries.


Problem: 
{problem}

# Incorrect answer:
{answer} 
</code>
Error or Output of the code: 
{output}
Corrected Answer: 
<code>
"""


# Problem: The original problem statement
# Incorrect answer: Possible solution to the problem 
# Error or Output of the code: Output generated using the possible answer
# Modified Answer: Modify the answer to address the error stated above


# modify_answer_prompt_template = """
# Modify the code given the Error

# Code: The original code
# Error/Incorrect Output: The error produced after execution or the output
# Modified Code: The corrected code which fixes the error. Import any library which is necessary

# Code: {answer}
# Error/Incorrect Output: {output}
# Modified Code: 
# {problem}
# """

# class ReactSignature(dspy.Signature):
#     __doc__ = """The following is a Data Science python code generation problem.
# You are provided with one possible answer to the problem
# and the output of the code using the provided answer.
# You need to determine if the answer is correct or not based on the input. 
# If the answer is NOT correct you need to provide a modified answer."""

#     problem = dspy.InputField(desc="The original problem statements")
#     possible_answer = dspy.InputField(desc="Possible solution to the problem")
#     generated_output = dspy.InputField(desc="Output generated using the possible answer")
#     is_correct = dspy.OutputField(desc="Is the answer correct or not. need to be YES or NO")
#     modified_answer = dspy.OutputField(desc="If the answer is not correct provide a modified answer")

    
# class ReactModule(dspy.Module):
#     def __init__(self, max_iterations=4):
#         super().__init__()
        
#         self.react = dspy.Predict(ReactSignature)
#         self.max_iterations = max_iterations

#     def forward(self, problem, possible_answer, generated_output):
        
#         response = self.react(problem=problem, 
#                               possible_answer=possible_answer, 
#                               generated_output=generated_output)
        
#         return dspy.Prediction(problem=problem, 
#                               possible_answer=possible_answer, 
#                               generated_output=generated_output,
#                               is_correct=response.is_correct,
#                               modified_answer=response.modified_answer)
    

# react_prompt = PromptTemplate.from_template("""
# Answer the following questions as best you can. You have access to the following tools:

# {tools}

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat 3 times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!

# Question: {input}
# Thought:{agent_scratchpad}
# """)

# llm = AzureOpenAI(api_key=os.environ["OPENAI_API_KEY"], 
#                   api_version=os.environ["OPENAI_API_VERSION"], 
#                   azure_deployment=os.environ["OPENAI_DEPLOYMENT"], 
#                   azure_endpoint=os.environ["OPENAI_ENDPOINT"])

# tools = [PythonREPLTool(verbose=True)]

# agent = create_react_agent(llm, tools, react_prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, 
#                                return_intermediate_steps=False,
#                               max_iterations=3,
#                               handle_parsing_errors=False)

In [10]:
from copy import copy
import tempfile
from pathlib import Path
from subprocess import Popen, PIPE
import threading

class Command(object):
    """
    This object takes in command and executes it with time out
    """

    def __init__(self, cmd):
        self.cmd = cmd
        self.process = None

    def run(self, timeout, file):
        def target():
            # Check if in Windows https://stackoverflow.com/questions/1325581/how-do-i-check-if-im-running-on-windows-in-python
            if os.name == "nt":
                self.process = Popen(self.cmd, shell=True, stdout=file, stderr=file)
            else:
                self.process = Popen(
                    self.cmd, shell=True, stdout=file, stderr=file, preexec_fn=os.setsid
                )
            self.process.communicate()

        thread = threading.Thread(target=target)
        thread.start()

        thread.join(timeout)
        if thread.is_alive():
            # Check if in Windows https://stackoverflow.com/questions/1325581/how-do-i-check-if-im-running-on-windows-in-python
            if os.name == "nt":
                Popen("TASKKILL /F /PID {pid} /T".format(pid=self.process.pid))
            else:
                os.killpg(self.process.pid, signal.SIGTERM)
            thread.join()
        
        return self.process.returncode 

def execute_code(code):
        
    with tempfile.TemporaryDirectory() as tempdir_name:
        # clean everything in the tempdir (unlikely)
        for file_name in os.listdir(tempdir_name):
            os.remove(file_name)

        tempdir_name = Path(tempdir_name)
        with open(tempdir_name / "program.py", "w", encoding="UTF-8") as f:
            f.write(code)

        cmd_text = "python {}".format(tempdir_name / "program.py")
        time_limit = 60  # should not change the official time_limit
        cmd = Command(
            cmd_text,
        )
        std_out_err_file = open(tempdir_name / "output.txt", "wt")
        exit_code = cmd.run(timeout=time_limit, file=std_out_err_file)  # 0 if there is no error
        std_out_err_file.close()
        std_out_err_file = open(tempdir_name / "output.txt", "r")
        output_text = std_out_err_file.read()
        std_out_err_file.close()
        
        with open(tempdir_name / "output.txt", "rt") as f:
            output_text = f.read()
            
        return exit_code, output_text

def run_react_pipeline(prob_prompt, max_iterations=3):
    
    
    
        
    
    def inference_helper(prompt, stop=["</code>", "# SOLUTION END"]):
        res = client.completions.create(
                    model=os.environ["OPENAI_DEPLOYMENT"],
                    prompt=prompt,
                    temperature=0.0,
                    max_tokens=1024,
                    top_p=0.95,
                    frequency_penalty=0,
                    presence_penalty=0,
                    stop=stop,
                    logprobs=1,
                )

        return res
    
    def sanitize_code(code):
        if("<code>" in code):
            code = code.split("<code>")[1]
        if("</code>" in code):
            code = code.split("</code>")[0]
        
        return code
    
    def get_code_to_execute(answer_code):
        code_to_execute = copy(prob_prompt)
        if("<code>" in code_to_execute):
            code_to_execute = code_to_execute.split("<code>")[1]
            
        if("</code>" in code_to_execute):
            code_to_execute=code_to_execute.split("</code>")[0]
            
        code_to_execute += answer_code
        last_operand = answer_code.strip().split('\n')[-1]
        if("=" in last_operand):
            last_operand = last_operand.split("=")[0].strip()
            code_to_execute += "print({})".format(last_operand)
        
        return code_to_execute
    
    response = inference_helper(prob_prompt)
    
    code_response = response.choices[0].text
    code_to_execute = get_code_to_execute(code_response)
    
    execution_status_code, execution_output = execute_code(code_to_execute)
    
    for i in range(0, max_iterations):
        
        print("code response", code_response)
        print("CODE to EXE", code_to_execute)
        print("EXECUTION OUTPUT", execution_status_code, execution_output)
        # check if the last execution is correct or not
        
        is_correct_prompt = is_correct_prompt_template.format(problem=prob_prompt, 
                                                          answer=code_response, 
                                                          output=execution_output)

        is_correct_response = inference_helper(is_correct_prompt, stop=['\n'])

        is_correct = is_correct_response.choices[0].text
        
        # continue if the last execution is not correct
        
        print("==========Correctness check", is_correct)
        
        if(is_correct.strip().lower() in ["no"]):
            # get modification
            modify_answer_prompt = modify_answer_prompt_template.format(problem=prob_prompt,
                                                                       answer=code_response,
                                                                       output=execution_output,
                                                                       is_correct=is_correct)
            
            modify_answer_response = inference_helper(modify_answer_prompt)
            
            # execute the modification 
            code_response = sanitize_code(modify_answer_response.choices[0].text)
            
            code_to_execute = get_code_to_execute(code_response)
            
            execution_status_code, execution_output = execute_code(code_to_execute)
            
        else:
            # stop the execution
            break
            
    return code_response, code_to_execute, is_correct, execution_output
    
    

In [11]:
from ds1000 import DS1000Problem

prob = DS1000Problem('./ds1000_data/Pandas/Completion/q3/')
print(prob["prompt"])
final_answer = run_react_pipeline(prob["prompt"])

Problem:
I have following pandas dataframe :


import pandas as pd
from pandas import Series, DataFrame
data = DataFrame({'Qu1': ['apple', 'potato', 'cheese', 'banana', 'cheese', 'banana', 'cheese', 'potato', 'egg'],
              'Qu2': ['sausage', 'banana', 'apple', 'apple', 'apple', 'sausage', 'banana', 'banana', 'banana'],
              'Qu3': ['apple', 'potato', 'sausage', 'cheese', 'cheese', 'potato', 'cheese', 'potato', 'egg']})


I'd like to change values in columns Qu1,Qu2,Qu3 according to value_counts() when value count great or equal 3
For example for Qu1 column
>>> pd.value_counts(data.Qu1) >= 3
cheese     True
potato    False
banana    False
apple     False
egg       False


I'd like to keep values cheese, because each value has at least three appearances.
From values potato, banana, apple and egg I'd like to create value others
For column Qu2 no changes :
>>> pd.value_counts(data.Qu2) >= 3
banana     True
apple      True
sausage   False


The final result as in attached t

In [ ]:
# Problem:
# I have data of sample 1 and sample 2 (`a` and `b`) – size is different for sample 1 and sample 2. I want to do a weighted (take n into account) two-tailed t-test.
# I tried using the scipy.stat module by creating my numbers with np.random.normal, since it only takes data and not stat values like mean and std dev (is there any way to use these values directly). But it didn't work since the data arrays has to be of equal size.
# For some reason, nans might be in original data, and we want to omit them.
# Any help on how to get the p-value would be highly appreciated.
# A:
<code>
import numpy as np
import scipy.stats
a = np.random.randn(40)
b = 4*np.random.randn(50)
</code>
p_value = ... # put solution in this variable
BEGIN SOLUTION
<code>

In [58]:
from ds1000 import DS1000Problem

prob = DS1000Problem('./ds1000_data/Numpy/Completion/q60/')
print(prob["prompt"])
final_answer = run_react_pipeline(prob["prompt"])

Problem:
I have data of sample 1 and sample 2 (`a` and `b`) – size is different for sample 1 and sample 2. I want to do a weighted (take n into account) two-tailed t-test.
I tried using the scipy.stat module by creating my numbers with np.random.normal, since it only takes data and not stat values like mean and std dev (is there any way to use these values directly). But it didn't work since the data arrays has to be of equal size.
For some reason, nans might be in original data, and we want to omit them.
Any help on how to get the p-value would be highly appreciated.
A:
<code>
import numpy as np
import scipy.stats
a = np.random.randn(40)
b = 4*np.random.randn(50)
</code>
p_value = ... # put solution in this variable
BEGIN SOLUTION
<code>

code response # remove nans
a = a[~np.isnan(a)]
b = b[~np.isnan(b)]

# calculate weights
w_a = np.ones(len(a))/len(a)
w_b = np.ones(len(b))/len(b)

# perform t-test
t_stat, p_value = scipy.stats.ttest_ind(a, b, equal_var=False, weights=(w_a, w_b))

C

In [25]:
execute_code("""
import pandas as pd
import math

df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
# Using list comprehension to create new columns with exponential values
result = pd.DataFrame({"A": df["A"], "B": df["B"], "exp_A": [math.exp(x) for x in df["A"]], "exp_B": [math.exp(x) for x in df["B"]]})
print(result)
""")

(0,
 '   A  B      exp_A       exp_B\n0  1  4   2.718282   54.598150\n1  2  5   7.389056  148.413159\n2  3  6  20.085537  403.428793\n')

In [37]:
import math
import pandas as pd


df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
# Possible Answer: 
# Modified Answer: 
# Using list comprehension to create new columns with exponential values
result = pd.DataFrame({col: df[col] for col in df.columns})
result["exp_A"] = [math.exp(x) for x in result["A"]]
result["exp_B"] = [math.exp(x) for x in result["B"]]
 
print(result["exp_B"])

# python_repl.run(code_to_execute)

0     54.598150
1    148.413159
2    403.428793
Name: exp_B, dtype: float64


In [48]:
for lib in ["Numpy"]:
    for i, prob in enumerate(ds_data[lib]):
        print(lib, i)
        
        prob_prompt = prob["prompt"]
        print(prob_prompt)
        
        
        
        break
    break


Numpy 0
Problem:
How do I get the dimensions of an array? For instance, this is (2, 2):
a = np.array([[1,2],[3,4]])

A:
<code>
import numpy as np
a = np.array([[1,2],[3,4]])
</code>
result = ... # put solution in this variable
BEGIN SOLUTION
<code>



In [12]:
lm = dspy.AzureOpenAI(
    model="stic-isl-gpt35",
    api_key=os.getenv('OPENAI_API_KEY'),
    api_base=os.getenv('OPENAI_ENDPOINT'),
    model_type="chat",
    api_version=os.getenv('OPENAI_API_VERSION'))

lm("hello")

['Hello! How can I assist you today?']

In [94]:
python_repl.run("""
import math

df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})


df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
result = pd.DataFrame({"A": df["A"], "B": df["B"], "exp_A": [math.exp(x) for x in df["A"]], "exp_B": [math.exp(x) for x in df["B"]]})
print(result)

""")

'NameError("name \'math\' is not defined")'

In [89]:
import math